<a href="https://colab.research.google.com/github/Logesh001/OCR/blob/main/hyperparametertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers optuna

In [ ]:
!pip install datasets

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        # Load and preprocess your data here
        self.data = []  # Replace with your data loading code
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoded_item = self.tokenizer(item["text"], padding="max_length", truncation=True)
        encoded_item["labels"] = item["labels"]
        return encoded_item

# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define paths to your train and test data
train_data_dir = "/content/drive/MyDrive/Goodnotes_train/train"
test_data_dir = "/content/drive/MyDrive/Goodnotes_test/test"

def load_your_train_data(data_dir):
    train_dataset = MyDataset(data_path=data_dir, tokenizer=tokenizer)
    return train_dataset

def load_your_test_data(data_dir):
    test_dataset = MyDataset(data_path=data_dir, tokenizer=tokenizer)
    return test_dataset


In [ ]:
import optuna
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

# Define the model configuration and tokenizer
model_name = "bert-base-uncased"
model_config = BertForSequenceClassification.config_class.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define paths to train and test datasets
train_data_dir = "/content/drive/MyDrive/Goodnotes_train/train"
test_data_dir = "/content/drive/MyDrive/Goodnotes_test/test"

# Define the hyperparameters to tune
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])

    # Set up the model and training arguments
    model = BertForSequenceClassification(config=model_config)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",  # Set the save strategy to "epoch"
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=3,
        load_best_model_at_end=True,
    )

    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=load_your_train_data(train_data_dir),  # Implement this function to load your train dataset
        eval_dataset=load_your_test_data(test_data_dir),    # Implement this function to load your test dataset
    )

    # Train the model
    trainer.train()

    # Evaluate on the test set
    results = trainer.evaluate()

    return results["eval_accuracy"]

# Create an Optuna study and run hyperparameter search
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # Change n_trials as needed

# Print the best hyperparameters and result
best_trial = study.best_trial
print("Best Trial:")
print("Value: ", best_trial.value)
print("Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")


[I 2023-08-31 12:15:23,996] A new study created in memory with name: no-name-3a064d50-5e70-4e9b-a389-eb23722293d5
[W 2023-08-31 12:15:27,668] Trial 0 failed with parameters: {'learning_rate': 0.00011159408733185579, 'batch_size': 8} because of the following error: ValueError('num_samples should be a positive integer value, but got num_samples=0').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-32-860ba9593594>", line 44, in objective
    trainer.train()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1555, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1569, in _inner_training_loop
    train_dataloader = self.get_train_dataloader()
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 837, in get_train_dataloader
   

ValueError: ignored